In [1]:
import logging
logging.basicConfig(filename = 'U_N_AV_CS.txt', filemode='w', level=logging.INFO)

import numpy as np
import os

from collections import defaultdict
from keras.optimizers import RMSprop
from keras.utils import plot_model
from pyfiction.agents.ssaqn_agent import SSAQNAgent
from pyfiction.agents.ssaqn_agent import load_embeddings
from pyfiction.simulators.games.catsimulator2016_simulator import CatSimulator2016Simulator
from pyfiction.simulators.text_games.simulators.MySimulator import StoryNode

logger = logging.getLogger(__name__)

"""
An example SSAQN agent for Saving John that uses online learning and prioritized sampling
"""

# Create the agent and specify maximum lengths of descriptions (in words)
agent = SSAQNAgent(train_simulators=CatSimulator2016Simulator(), mode = 'Average')

# Learn the vocabulary (the function samples the game using a random policy)
agent.initialize_tokens('vocabulary.txt')

optimizer = RMSprop(lr=0.0001)

embedding_dimensions = 300
lstm_dimensions = 32
dense_dimensions = 8

agent.create_model(embedding_dimensions=embedding_dimensions,
                   lstm_dimensions=lstm_dimensions,
                   dense_dimensions=dense_dimensions,
                   optimizer=optimizer,
                  embeddings=None,
                  embeddings_trainable=None)
    

# Visualize the model
try:
    plot_model(agent.model, to_file='model.png', show_shapes=True)
except ImportError as e:
    logger.warning("Couldn't print the model image: {}".format(e))

# Iteratively train the agent on a batch of previously seen examples while continuously expanding the experience buffer
# This example seems to converge to the optimal reward of 19.4
epochs = 1
for i in range(epochs):
    logger.info('Epoch %s', i)
    agent.train_online(episodes=256, batch_size=64, gamma=0.95, epsilon_decay=0.99, prioritized_fraction=0.25)


Using TensorFlow backend.
/Users/francescopreta/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


---------------
Complete model:
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_state (InputLayer)         (None, None)          0                                            
____________________________________________________________________________________________________
input_action (InputLayer)        (None, None)          0                                            
____________________________________________________________________________________________________
embedding_shared (Embedding)     (None, None, 300)     109500      input_state[0][0]                
                                                                   input_action[0][0]               
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 300)           0   